In [33]:
import pandas as pd

# Load the data
df = pd.read_csv("clean_forum_dataset.csv", encoding='ISO-8859-1') # Replace with your file path
df = df.sample(frac = 0.001, random_state = 333)
#print(len(test_df))
print(len(df))
df.head()

5


,Unnamed: 0,topic_id,translated_raw
754,755,145,I am considering embedding a WIZ140SR into an ...
619,620,1296,"Hi,\r\n\r\nI will try to describe how to use D..."
2390,2395,3589,"Among the github code\r\n // pb_05, pb_12 p..."
2025,2026,2885,hello.\r\n\r\nInquiries to Serial Communicatio...
3593,3600,6659,I want to implement DHCP using the W6100.\r\n\...


In [34]:
import requests
import json

# 제공된 API 키
MY_API_KEY = "087acfa4-6ba7-4b61-8f7d-de54e68b9777"
CHATBOT_ID = "RPPmiqEYrxmbXsAfLhg0I"
URL = "https://www.chatbase.co/api/v1/chat"

In [35]:
def send_request(chat_data):
    headers = {
        "Authorization": f"Bearer {MY_API_KEY}",
        'Content-Type': 'application/json'
    }
    data = {
        "messages": [
            {"content": chat_data, "role": "user"}
        ],
        "chatbotId": CHATBOT_ID,
        "stream": False,
        "temperature": 0,
        "model": "gpt-3.5-turbo"
    }

    try:
        response = requests.post(URL, headers=headers, data=json.dumps(data))
        if response.status_code == 200:
            json_data = response.json()
            return json_data['text']
        else:
            print('Error: HTTP Status Code', response.status_code)
            print('Response content:', response.text)
    except json.JSONDecodeError as e:
        print('JSONDecodeError:', e)
        print('Raw response:', response.text)


In [36]:
df['refine_raw'] = None
for index, row in df.iterrows():
    try:
        response = send_request(row['translated_raw'])  # send_request 함수로 요청 보내고 응답 받기
        
        df.at[index, 'refine_raw'] = response  # 응답을 DataFrame에 저장
        if (index + 1) % 10 == 0:  # 10번째 마다 실행
            print(f"진행된 개수: {index + 1}")  # 현재까지의 응답 배열 출력
            df_partial = df.loc[:, ~df.columns.str.contains('^Unnamed')]

            # Remove rows where 'refine_raw' is None
            df_partial = df_partial[df_partial['refine_raw'].notna()]

            # Select only the relevant columns
            df_partial = df_partial[['topic_id', 'translated_raw', 'refine_raw']]
            df_partial.to_csv(f"forum_dataset_reduced.csv")
    except Exception as e:
        print(f"에러 발생: {e}")
        print(f"현재까지 진행된 개수: {index}") 
        break

# Save the final dataframe
df_final = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df_final = df_final[df_final['refine_raw'].notna()]
df_final = df_final[['topic_id', 'translated_raw', 'refine_raw']]
df_final.to_csv("forum_dataset_final.csv")


진행된 개수: 619
진행된 개수: 2025
진행된 개수: 3593
